In [1]:
import cv2 
import numpy as np

Fonction pour le diamètre de la pièce de monnaie

In [2]:
def conversion_piece(image_path,taille_piece):
    image = cv2.imread(image_path)#charge image
    if image is None:
        raise ValueError("L'image n'a pas pu être chargée.")
    
    taille_noyau = (5, 5)
    sigma = 0
    blurred_image = cv2.GaussianBlur(image, taille_noyau, sigma)
    
    
    gray = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    thresh = cv2.bitwise_not(thresh)

    element = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(5, 5))

    morph_img = thresh.copy()
    cv2.morphologyEx(src=thresh, op=cv2.MORPH_CLOSE, kernel=element, dst=morph_img)

    contours,_ = cv2.findContours(morph_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #trouver les contours

    output_image = cv2.cvtColor(morph_img, cv2.COLOR_GRAY2BGR)# préparer l'image de sortie

    height, width = morph_img.shape[:2]# dimensions de l'image

    y_limit1 = height - 800 #limite
    y_limit2 = 800

    max_contour = None #contour et aire
    max_area = 0

    for cnt in contours: #on cherche dans la zone les contours
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        if (center[1] > y_limit1 or center[1] < y_limit2):
            area = cv2.contourArea(cnt)
            if area > max_area:
                max_area = area
                max_contour = cnt
                
    if max_contour is not None:
        (x, y), radius = cv2.minEnclosingCircle(max_contour)
        diameter = 2 * int(radius)
    else:
        diameter = 0

    return taille_piece/diameter #diametre

Brouillon test des fonctions OpenCV pour la pièce de monnaie

In [3]:
image_path = '../DataBase/test1.jpg'
conversion = conversion_piece(image_path,22.6)
print(f"Estimated in pixels: {conversion}")

Estimated in pixels: 0.005937992643194956


In [4]:
image_path = '../DataBase/image3.jpg'
image = cv2.imread(image_path)

In [4]:
taille_noyau = (5, 5)
sigma = 0
image = cv2.GaussianBlur(image, taille_noyau, sigma)

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
th = cv2.adaptiveThreshold(gray_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,71,17)
edges = cv2.adaptiveThreshold(th, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 3, 2)

NameError: name 'image' is not defined

In [ ]:
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

output_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

In [1]:
height = edges.shape[0]
width = edges.shape[1]

x_limit = width
y_limit = height - 800
top_left = (0, y_limit)
bottom_right = (x_limit, height)
cv2.rectangle(output_image, top_left, bottom_right, (255, 0, 0), 2)

y_limit2 = 800
top_left2 = (0, 0)
bottom_right2 = (x_limit, y_limit2)
cv2.rectangle(output_image, top_left2, bottom_right2, (255, 0, 0), 2)

max_contour = None
max_area = 0

for cnt in contours:
    (x, y), radius = cv2.minEnclosingCircle(cnt)
    center = (int(x), int(y))
    radius = int(radius)
    if (center[1] > y_limit or center[1] < y_limit2):
        area = cv2.contourArea(cnt)
        if area > max_area:
            max_area = area
            max_contour = cnt
            
if max_contour is not None:
    (x, y), radius = cv2.minEnclosingCircle(max_contour)
    center = (int(x), int(y))
    radius = int(radius)
    diameter = 2 * radius
    cv2.circle(output_image, center, radius, (0, 255, 0), 2)
    cv2.circle(output_image, center, 1, (0, 0, 255), 3)
    start_point = (int(x - radius), int(y))
    end_point = (int(x + radius), int(y))
    cv2.line(output_image, start_point, end_point, (255, 0, 0), 2)

cv2.imwrite('output_image_test.png', output_image)
print(f"Estimated diameter in pixels: {diameter}")

NameError: name 'edges' is not defined